<a href="https://colab.research.google.com/github/zozunin/bermuda/blob/master/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [7]:
!pip install pymorphy2==0.8
!pip install spacy==2.0.12

     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 7.1MB 22.9MB/s 
     |████████████████████████████████| 22.0MB 1.4MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
     |████████████████████████████████| 1.2MB 54.4MB/s 
     |████████████████████████████████| 194kB 61.9MB/s 
     |████████████████████████████████| 153kB 48.0MB/s 
     |████████████████████████████████| 604kB 47.1MB/s 
     |████████████████████████████████| 450kB 57.9MB/s 
  Created wheel for spacy: filename=spacy-2.0.12-cp36-cp36m-linux_x86_64.whl size=29062671 sha256=e896bc93730c03e833258f2d890cc43fcf86dc30e7db690f5699a651c455e1ff
  Stored in directory: /root/.cache/pip/wheels/60/0b/bb/7c2e28db574dbb2358176934eddd32a1c5f838ba0bc23eaaab
  Created wheel for thinc: filename=thinc-6.10.3-cp36-cp36m-linux_x86_64.whl size=4121232 sha256=9627d2ca4177ccf8a9927aeaaddefeaca7619405583a37ea970d7fa5ba19e45f
  Stored in directory: /root/.cache/pip/wheels/db/bc/e1/9b321b6b203

In [8]:
!git clone https://github.com/buriy/spacy-ru.git #&& cp -r ./spacy-ru/ru2/. /content/gdrive/My Drive/Colab Notebooks/Бермуды/ru2

Cloning into 'spacy-ru'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 231 (delta 7), reused 29 (delta 7), pack-reused 191
Receiving objects: 100% (231/231), 99.21 MiB | 29.39 MiB/s, done.
Resolving deltas: 100% (71/71), done.


In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk_stop_words = set(stopwords.words("russian"))
import re

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
import spacy
nlp = spacy.load('spacy-ru/ru2')

In [0]:
import sqlite3

con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/districts_posts(1).db')

In [0]:
for_proc = []
with con:    
    cur = con.cursor()    
    cur.execute("SELECT * FROM districts")
    rows = cur.fetchall()
 
    for row in rows:
        for_proc.append(row)

In [0]:
con.close()

In [5]:
len(for_proc)

13965

In [0]:
def prepro(text):
    text = re.sub('[.,\?!;:]', ' ; ', text)
    spacy_step = nlp(text)
    temp = [t.lemma_ for t in spacy_step if bool(re.match(r'[а-яА-Я]+$|[а-яА-Я]+\-[а-яА-Я]+$', t.lemma_)) and t.lemma_ not in nltk_stop_words and t.lemma_ != '-PRON-']
    return ' '.join(temp)

In [0]:
from tqdm import tqdm_notebook

In [15]:
updated = []
for post in tqdm_notebook(for_proc):
    post += tuple([prepro(post[2])])
    updated.append(post)

In [16]:
updated[0]

(0,
 'автозавод',
 'Уважаемые подписчики! \nУ нас, для Вас , теперь есть инстаграм! https://instagram.com/podslushano_avtozavodnn?igshid=15iwguo5mcjzx\nПодпишись на нас✅\nТак же , работу возобновила вторая группа «ищу тебя». Все посты о поисках чего-либо писать ✍️ в https://vk.com/club114783231 \nЗа репост огромное спасибо 🙏🏻 📌',
 2,
 0,
 0,
 1,
 0,
 33333,
 11,
 21,
 0,
 32,
 '2019-08-05',
 'уважаемые подписчик инстаграм подпишись работа возобновила группа искать весь пост поиск либо писать репост огромный спасибо')

In [0]:
year_cl = []
for post in updated:
    date = post[-2].split('-')
    if date[0]=='2019' and date[1] in ['01', '02', '03', '04', '05', '06', '07', '08']:
        year_cl.append(post)
    elif date[0] == '2018' and date[1] in ['09', '10', '11', '12']:
        year_cl.append(post)
    else:
        pass


In [18]:
len(year_cl)

5696

In [0]:
import sqlite3
from sqlite3 import Error

def sql_connection():
    try:
        con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/posts_per_year.db')
        return con
    except Error:
        print(Error)

def sql_table(con):
    cursorObj = con.cursor()
    cursorObj.execute("""CREATE TABLE districts(id integer PRIMARY KEY, district text, post text, cleaned_post text, photo integer, audio integer, video integer, link integer, 
                        poll integer, views integer, comments integer, likes integer, share integer, total_action integer, date text)""")
    con.commit()

con = sql_connection()
sql_table(con)

In [24]:
import sqlite3

con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/posts_per_year.db')

def sql_insert(con, entities):
    cursorObj = con.cursor()
    cursorObj.execute('''INSERT INTO districts(id, district, post, cleaned_post, photo, audio, video, link, poll, views, comments, 
                        likes, share, total_action, date) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', entities)
    con.commit()

for post in tqdm_notebook(year_cl):
    if post[-1] != '':
        entities = post[0:3]+tuple([post[-1]])+post[3:-1]
        sql_insert(con, entities)

In [0]:
con.close()

In [26]:
len(year_cl)

5696

In [0]:
import sqlite3

con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/posts_per_year.db')

In [0]:
for_del = []
with con:    
    cur = con.cursor()    
    cur.execute("SELECT * FROM districts")
    rows = cur.fetchall()
 
    for row in rows:
        for_del.append(row)

In [0]:
con.close()

In [0]:
texts = {}
for post in for_del:
    if post[2] not in texts:
        texts[post[2]] = [post[0]]
    else:
        texts[post[2]] += [post[0]]


In [32]:
for k, v in texts.items():
    if len(v) > 1:
        print(v[0], '\n', k)

579 
 Всем привет) 
Ребят,кто поступил в институт пищевых технологий,на товароведа после 9 на платку?)
Ставьте +)
Познакомимся) 
Админа всего хорошего) 
Анон)
591 
 Может лежит у кого нибудь без дела такой столик? Пялиться где нибудь)))) 
Ищу )))))
604 
 Доброго времени суток
Кто нибудь сдаёт 1-ку или студию ( может комнату ) 
Молодой человек, студент 
Работаю официально, есть стабильный заработок
Все предложения рассмотрю, лайк или комментарии 
Анонимно
629 
 Всем привет! 
Меня зовут Дмитрий Смирнов, и я приглашаю вас посетить сеанс профессионального массажа! 
 
Расскажу коротко о себе: 
Свою деятельность начал в 2010 году, получив диплом квалифицированного специалиста в области медицинского массажа. С того момента и на протяжении 7 лет непрерывно работаю в сфере массажа и регулярно прохожу курсы по повышению квалификации. Количество клиентов, обратившихся ко мне за весь период моей профессиональной деятельности, составляет более 5000 человек! 
 
Массаж – это средство поддержания здор

In [33]:
avoid = [579, 591,604, 693,706, 932, 945, 969, 1200, 1231, 1319, 1381, 1410, 1492, 1782, 1790, 1909, 2048, 2057, 2064, 2208, 2279, 2328, 2394, 2484, 2489, 2517, 2636, 2637, 2648, 2704,
         2767, 2810, 2814, 2826, 2839, 2870, 2931, 3034, 3120, 3281, 3343, 3388, 3394,3419, 3437, 5685, 5802, 6328, 6505, 6608, 11182]
len(avoid)

52

In [0]:
ids = []
for k, v in texts.items():
    if len(v) > 1:
        ids+=v

In [0]:
ids += [0, 549, 558, 575, 583, 598, 624, 629, 633, 638, 642, 645, 659, 664, 668, 673, 680, 690, 692, 699, 710, 715, 718, 720, 726, 730, 731, 741, 744, 753, 763, 767, 778, 775, 774, 785, 798, 799, 800, 807, 805, 811, 817, 823, 824, 826, 827, 829, 830, 836, 838, 849, 851, 861, 865, 876, 884, 886, 890, 899, 905, 910, 919, 920, 922, 934, 935, 944, 947, 957, 963, 973, 985, 990, 1003, 1025, 1156, 1369, 1479]

In [0]:
unique = []
for post in for_del:
    if post[0] not in ids:
        unique.append(post)
    if post[0] in avoid:
        unique.append(post)


In [39]:
len(unique)

4264

In [40]:
unique[1]

(551,
 'автозавод',
 'Ищу девушку с которой ехал на 46 маршрутке,30.08 в 06:00. На тебе были серые штаны,коричневая куртка и розово-голубые волосы.Ты должна узнать кто я. Хотел бы пообщаться. Лайкни пост. \nАнон.😉',
 'искать девушка которой ехать маршрутка серый штаны коричневый куртка розовый голубой волос должный узнать хотеть пообщаться лайкни пост анон',
 0,
 0,
 0,
 0,
 0,
 1989,
 0,
 3,
 0,
 3,
 '2019-08-30')

In [0]:
import sqlite3
from sqlite3 import Error

def sql_connection():
    try:
        con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/dCleaned.db')
        return con
    except Error:
        print(Error)

def sql_table(con):
    cursorObj = con.cursor()
    cursorObj.execute("""CREATE TABLE districts(id integer PRIMARY KEY, district text, post text, cleaned_post text, photo integer, audio integer, video integer, link integer, 
                        poll integer, views integer, comments integer, likes integer, share integer, total_action integer, date text)""")
    con.commit()

con = sql_connection()
sql_table(con)

In [41]:
import sqlite3
from tqdm import tqdm_notebook

con = sqlite3.connect('/content/gdrive/My Drive/study/bermuda/dCleaned.db')

def sql_insert(con, entities):
    cursorObj = con.cursor()
    cursorObj.execute('''INSERT INTO districts(id, district, post, cleaned_post, photo, audio, video, link, poll, views, comments, 
                        likes, share, total_action, date) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', entities)
    con.commit()

for post in tqdm_notebook(unique):
    sql_insert(con, post)
    